# [Going Deeper NLP 1] SentencePiece

Google에서 제공하는 오픈소스 기반 Sentence Tokenizer/Detokenizer 패키지인 SentencePiece를 사용하는 프로젝트입니다. 

---

## 프로젝트 목표
---
- 한국어로 구성된 네이버 영화리뷰 감정분석 문제에 SentencePiece를 적용한다. 
- Unet, generator, discriminator레이어를 구성한다.


## 프로젝트 설명
---
1. 네이버 영화리뷰 감정분석 코퍼스에 sentencepiece를 적용시킨 모델 학습하기

2. 학습된 모델로 sp_tokenize() 메소드 구현하기

3. 구현된 토크나이저를 적용하여 네이버 영화리뷰 감정분석 모델을 재학습하기

4. KoNLPy 형태소 분석기를 사용한 모델과 성능 비교하기

5. (보너스) SentencePiece 모델의 model_type, vocab_size 등을 변경해 가면서 성능 개선 여부 확인하기

6. Word Vector는 활용할 필요가 없습니다. 활용이 가능하지도 않을 것입니다.

7. 머지않아 SentencePiece와 BERT 등의 pretrained 모델을 함께 활용하는 태스크를 다루게 될 것입니다.

## 1. 데이터 불러오기 및 전처리

In [3]:
import pandas as pd
import sentencepiece as spm
import urllib.request
import csv

In [4]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7fa160346310>)

In [6]:
naver_df = pd.read_table('ratings.txt')
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [7]:
print('데이터 개수 :', len(naver_df))

데이터 개수 : 200000


**결측치 확인 및 제거**

In [9]:
naver_df.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [13]:
naver_df = naver_df.dropna(how='any')
print(naver_df.isnull().sum)

<bound method NDFrame._add_numeric_operations.<locals>.sum of            id  document  label
0       False     False  False
1       False     False  False
2       False     False  False
3       False     False  False
4       False     False  False
...       ...       ...    ...
199995  False     False  False
199996  False     False  False
199997  False     False  False
199998  False     False  False
199999  False     False  False

[199992 rows x 3 columns]>


In [15]:
print('결측치 제거 후 데이터 개수 :', len(naver_df))

결측치 제거 후 데이터 개수 : 199992


In [16]:
naver_df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [17]:
type(naver_df['document'])

pandas.core.series.Series

In [ ]:
import sentencepiece as spm



In [ ]:
def tokenize(corpus):  # corpus: Tokenized Sentence's List
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    tokenizer.fit_on_texts(corpus)

    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, tokenizer

In [20]:
import sentencepiece as spm
import os
temp_file = os.getenv('HOME')+'/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp'
type(temp_file)
with open(temp_file, 'r') as f:
    f.read()

# vocab_size = 8000

# with open(temp_file, 'w') as f:
#     for row in filtered_corpus:   # 이전 스텝에서 정제했던 corpus를 활용합니다.
#         f.write(str(row) + '\n')

# spm.SentencePieceTrainer.Train(
#     '--input={} --model_prefix=korean_spm --vocab_size={}'.format(temp_file, vocab_size)    
# )
# #위 Train에서  --model_type = 'unigram'이 디폴트 적용되어 있습니다. --model_type = 'bpe' 로 옵션을 주어 변경할 수 있습니다.

# !ls -l korean_spm*

FileNotFoundError: [Errno 2] No such file or directory: '/home/aiffel-dj26/aiffel/sp_tokenizer/data/korean-english-park.train.ko.temp'

In [ ]:
s = spm.SentencePieceProcessor()
s.Load('korean_spm.model')

# SentencePiece를 활용한 sentence -> encoding
tokensIDs = s.EncodeAsIds('아버지가방에들어가신다.')
print(tokensIDs)

# SentencePiece를 활용한 sentence -> encoded pieces
print(s.SampleEncodeAsPieces('아버지가방에들어가신다.',1, 0.0))

# SentencePiece를 활용한 encoding -> sentence 복원
print(s.DecodeIds(tokensIDs))

In [ ]:
def sp_tokenize(s, corpus):

    tensor = []

    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    with open("./korean_spm.vocab", 'r') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]

        word_index.update({idx:word})
        index_word.update({word:idx})

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word

In [ ]:
#sp_tokenize(s, corpus) 사용예제

my_corpus = ['나는 밥을 먹었습니다.', '그러나 여전히 ㅠㅠ 배가 고픕니다...']
tensor, word_index, index_word = sp_tokenize(s, my_corpus)
print(tensor)

# 루브릭 평가
---
1. **pix2pix 모델 학습을 위해 필요한 데이터셋을 적절히 구축하였다.**   

    - 자르기, 뒤집기, 회전을 적용해 데이터셋을 구축했다. 


2. **pix2pix 모델을 구현하여 성공적으로 학습 과정을 진행하였다.**

    - U-Net generator와 discriminator 모델을 구현하였고, train_step은 안정적으로 진행되었다. 
     
     
3. **학습 과정 및 테스트에 대한 시각화 결과를 제출하였다.** 

    - 10 epochs를 학습했고, 도로에 대한 segmentation이 어느정도 잘 생성되었다. 
    - 5장의 validation 이미지를 시각화해 결과를 확인했다. 

    
    
# 회고
---

## 다짐
- U-Net으로 인코딩, 디코딩하는 모델구조가 항상 흥미로웠는데 마지막 프로젝트로 진행할 수 있어서 재미있었다. 또한 이전에 GAN을 공부할 때, 단순히 무작위의 노이즈를 입력으로 넣어서 생성하는 부분이 잘 납득이 가지 않았었는데 이런 점을 벌써 해소해준 Pix2Pix 모델이 있어서 놀랐다. 딥러닝, 머신러닝에서는 어떤 아키텍쳐를 만났을 때 의문점을 갖고 더 발전시킬 수 있는 방안에 대해서 짧게나마 고민하는 방법이 필요하다고 생각한다.